In [2]:
import string
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

In [3]:
df = pd.read_csv('twitter_training.csv')
df.head()

,2401,Borderlands,Positive,"im getting on borderlands and i will murder you all ,"
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [4]:
df = df.rename({"Positive": "sentiment"}, axis=1)
df = df.rename({"im getting on borderlands and i will murder you all ,": "text"}, axis=1)



In [5]:
df.head()

,2401,Borderlands,sentiment,text
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [6]:
df.value_counts()

2401   Borderlands                        sentiment   text                                                                                                                           
5810   HomeDepot                          Positive    Congratulations!!                                                                                                                  5
2895   Dota2                              Positive    Wow                                                                                                                                5
10181  PlayerUnknownsBattlegrounds(PUBG)  Irrelevant  Really                                                                                                                             5
3108   Dota2                              Negative    wtf                                                                                                                                5
1932   CallOfDutyBlackopsColdWar          Positive    Wow             

In [7]:
def remove_pun_and_numbers(text):
    # Combine punctuation and digits
    text = str(text)
    to_remove = string.punctuation + string.digits
    return text.translate(str.maketrans('', '', to_remove))

In [8]:
print(df.columns)

Index(['2401', 'Borderlands', 'sentiment', 'text'], dtype='object')


In [9]:
df['text'] = df['text'].apply(remove_pun_and_numbers)
df = df[df['text'].notnull() & df['text'].apply(lambda x: isinstance(x, str))]


In [10]:
from sklearn.model_selection import train_test_split

In [11]:
df.text.head()

0    I am coming to the borders and I will kill you...
1    im getting on borderlands and i will kill you all
2    im coming on borderlands and i will murder you...
3    im getting on borderlands  and i will murder y...
4    im getting into borderlands and i can murder y...
Name: text, dtype: object

In [12]:
x = df.iloc[:,3]
y = df.iloc[:,2]

In [13]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = .2)

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [15]:
vec = TfidfVectorizer(ngram_range=(1, 2), analyzer='char')
x_train = [str(x) for x in x_train]


In [16]:
from sklearn import pipeline
from sklearn import linear_model

In [17]:
model_pipe = pipeline.Pipeline([('vec', vec), ('clf', linear_model.LogisticRegression())])

In [18]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

model_pipe = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('scaler', StandardScaler(with_mean=False)),
    ('logreg', LogisticRegression(max_iter=10000)),
])

model_pipe.fit(x_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('scaler', StandardScaler(with_mean=False)),
                ('logreg', LogisticRegression(max_iter=10000))])

In [19]:
model_pipe.classes_

array(['Irrelevant', 'Negative', 'Neutral', 'Positive'], dtype=object)

In [20]:
predict_val = model_pipe.predict(x_test)

In [21]:
from sklearn import metrics

In [22]:
metrics.accuracy_score(y_test,predict_val)

0.7922608288143537

In [23]:
metrics.confusion_matrix(y_test, predict_val)

array([[1920,  201,  209,  238],
       [ 182, 3684,  315,  349],
       [ 158,  325, 2877,  362],
       [ 181,  286,  297, 3353]])

In [31]:
model_pipe.predict(['sure'])

array(['Neutral'], dtype=object)

In [25]:
import pickle

In [26]:
new_file = open('model.pkl', 'wb')
pickle.dump(model_pipe, new_file)
new_file.close()